1. Parameter Accuracy

This part will check how accurate the LLM parse the parameters for the function call after reading the query.

2. delivery rate

In [4]:
import json
#delivery rate
def getDeliveryRate(model, numPlan):
    delivered = 0
    with open (f'Output/{model}/plans/toolUsePlans.jsonl','r') as f:
         plans = [json.loads(line.strip()) for line in f]

    for record in plans:
        if record['plan'] != "":
            delivered += 1
    return delivered / numPlan

In [8]:
deliverayRate = getDeliveryRate('llama318b',300)
deliverayRate

0.8466666666666667

In [32]:
def getInvalidLoopAndWrongOrderRate():
    for log in logs:
        for step in log['log']:
            if step['state'] != 'Successful':
                print(log['index'], step['step'],step['state'], step['action'])

11 3 Illegal args. Parameter Error Action 3: RestaurantSearch[Expensive Budget, US Cuisine, [Good Environment]]
11 4 Illegal args. Parameter Error Action 4: RestaurantSearch[Expensive Budget, US Cuisine, [Good Environment]]
11 5 same action 3 times repeated Action 5: RestaurantSearch[Expensive Budget, US Cuisine, [Good Environment]]
20 2 Illegal args. Parameter Error Action 2: RestaurantSearch[Good Value, Korean, [Good Flavor]]
20 3 Illegal args. Parameter Error Action 3: RestaurantSearch[Good Value, Korean, [Good Flavor]]
20 4 same action 3 times repeated Action 4: RestaurantSearch[Good Value, Korean, [Good Flavor]]
22 3 Illegal args. Parameter Error Action 3: RestaurantSearch[Expensive Budget, US Cuisine, [Good Environment, Good Flavor, Good Freshness]]
22 4 Illegal args. Parameter Error Action 4: RestaurantSearch[Expensive Budget, US Cuisine, [Good Environment, Good Flavor, Good Freshness]]
22 5 same action 3 times repeated Action 5: RestaurantSearch[Expensive Budget, US Cuisine, [G

In [66]:
#need revision to be more precise
def getInvalidLoopAndWrongOrderRate(model):
    allError = 0
    invalidArgDeadLoop = 0
    wrongOrder = 0

    stoppedList = []
    with open (f'Output/{model}/plans/toolUseLogs.jsonl','r') as f:
        logs = [json.loads(line.strip()) for line in f]
    for log in logs:
        for step in reversed(log['log']):
            if step['state'] != 'Successful':
                #print(log['index'], step['step'],step['state'], step['action'])
                if step['state'] == 'same action 3 times repeated':
                    allError += 1
                    stoppedList.append([log['index'],step['step']])

    for stop in stoppedList:
        plan = stop[0] - 1
        index = stop[1] - 2

        if logs[plan]['log'][index]['state'] == 'Illegal args. Parameter Error':
            invalidArgDeadLoop += 1
        elif logs[plan]['log'][index]['state'] == 'Illegal args. Other Error':
            wrongOrder += 1
        else:
            continue


    #print(stoppedList)
    return allError, invalidArgDeadLoop, wrongOrder

In [67]:
model = 'llama318b'
allError,invalidArgDeadLoop,wrongOrder = getInvalidLoopAndWrongOrderRate(model)
print(f"{model} have {allError} error stops. In them, {invalidArgDeadLoop} are dead loop due to invalid argument function call, {wrongOrder} are call the function in wrong order.")

llama318b have 46 error stops. In them, 22 are dead loop due to invalid argument function call, 23 are call the function in wrong order.


In [81]:
def getRightFunctionCall(model):
    rightRestaurantSearch = 0
    rightAccommodationSearch = 0
    rightAttractionSearch = 0
    with open (f'Output/{model}/plans/toolUseLogs.jsonl','r') as f:
        logs = [json.loads(line.strip()) for line in f]
    for log in logs:
        for step in reversed(log['log']):
            if step['state'] == 'Successful':
                if 'Accommodation' in step['action']:
                     rightAccommodationSearch += 1
                     break
        
        for step in reversed(log['log']):
            if step['state'] == 'Successful':
                if 'Restaurant' in step['action']:
                     rightRestaurantSearch += 1
                     break
        
        for step in reversed(log['log']):
            if step['state'] == 'Successful':
                if 'Attraction' in step['action']:
                     rightAttractionSearch += 1
                     break
    
    return rightAccommodationSearch, rightRestaurantSearch, rightAttractionSearch

if __name__ == "__main__":
    model = 'llama318b'
    acco, res, right = getRightFunctionCall(model)
    print(acco, res, right )

272 276 294


In [82]:
#case study

In [2]:
import json
from fuzzywuzzy import fuzz
import numpy as np
import sys
import os
import json
import re

def getID(name,address,category):
    #as long as there is a '-', then return -2
    #if there is an empty list, then return []
    #if the information doesn't match, return -1

    if name == "-" and address == "-":
        return -2

    #normal case
    idFromName = []
    idFromAddress = []

    address = address.split(",")[0]
    
    #restaurants
    if category == 'restaurants':
        for restaurant in restaurants:
            if restaurant['name'].lower() == name.lower():
                idFromName.append(restaurant['business_id'])
            if restaurant['address'].lower() == address.lower():
                idFromAddress.append(restaurant['business_id'])
        set1 = set(idFromName)
        set2 = set(idFromAddress)
        #if the extracted id from name and address make an agreement
        if(len(set1 & set2) == 1):
            return list(set1 & set2)[0]
        # if not, we have to use similarity score to determine the id
        else:
            name_sim_score = []
            address_sim_score = []

            for restaurant in restaurants:
                name_sim_score.append(fuzz.ratio(name.lower(), restaurant['name'].lower()))
                address_sim_score.append(fuzz.ratio(address.lower(), restaurant['address'].lower()))

            scores = np.array(name_sim_score) + np.array(address_sim_score)
            #if the score is high enough, then we claim the id
            if max(scores) >= 120:
                return restaurants[np.argmax(scores)]['business_id']
            #if the score is less than 60 for each, then we indicate that the business is out of the pool
            else:
                return -1
    #attractions 
    if category == 'attractions':
        for attraction in attractions:
            if attraction['name'].lower() == name.lower():
                idFromName.append(attraction['business_id'])
            if attraction['address'].lower() == address.lower():
                idFromAddress.append(attraction['business_id'])
        
        set1 = set(idFromName)
        set2 = set(idFromAddress)

        if(len(set1 & set2) == 1):
            return list(set1 & set2)[0]
        else:
            name_sim_score = []
            address_sim_score = []

            for attraction in attractions:
                name_sim_score.append(fuzz.ratio(name.lower(), attraction['name'].lower()))
                address_sim_score.append(fuzz.ratio(address.lower(), attraction['address'].lower()))

            if max(name_sim_score) == 100:
                return attractions[np.argmax(name_sim_score)]['business_id']

            scores = np.array(name_sim_score) + np.array(address_sim_score)
            if max(scores) >= 120:
                return attractions[np.argmax(scores)]['business_id']
            else:
                return -1
    #hotels
    if category == 'hotels':
        for hotel in hotels:
            if hotel['name'].lower() == name.lower():
                idFromName.append(hotel['business_id'])
            if hotel['address'].lower() == address.lower():
                idFromAddress.append(hotel['business_id'])
        set1 = set(idFromName)
        set2 = set(idFromAddress)
        if(len(set1 & set2) == 1):
            return list(set1 & set2)[0]
        else:
            name_sim_score = []
            address_sim_score = []

            for hotel in hotels:
                name_sim_score.append(fuzz.ratio(name.lower(), hotel['name'].lower()))
                address_sim_score.append(fuzz.ratio(address.lower(), hotel['address'].lower()))

            scores = np.array(name_sim_score) + np.array(address_sim_score)
            if max(scores) >= 120:
                return hotels[np.argmax(scores)]['business_id']
            else:
                return -1

def prepareEval(plan):
    plan_eval = []
    for days in plan['itinerary']:
        day = {}
        day['days'] = days['days']
        #print(days['breakfast']['name'])
        day['breakfast'] = getID(days['breakfast']['name'],days['breakfast']['address'],'restaurants')
        day['morning_attractions'] = [getID(attraction['name'],attraction['address'],'attractions') for attraction in days['morning_attractions']]
        day['lunch'] = getID(days['lunch']['name'],days['lunch']['address'],'restaurants')
        day['afternoon_attractions'] = [getID(attraction['name'],attraction['address'],'attractions') for attraction in days['afternoon_attractions']]
        day['dinner'] = getID(days['dinner']['name'],days['dinner']['address'],'restaurants')
        day['night_attractions'] = [getID(attraction['name'],attraction['address'],'attractions') for attraction in days['night_attractions']]
        day['accommodation'] = getID(days['accommodation']['name'],days['accommodation']['address'],'hotels')
        plan_eval.append(day)
    #print(plan_eval)
    return plan_eval



def populateCordinates(plan_eval, data, data_hotel):
    cordinates = []
    for day in plan_eval:
        cordinate_one_day = []

        #if the hotel is invalid, we skip the day
        if(day['accommodation'] == -1 or day['accommodation'] == -2):
            continue

        if(day['accommodation'] != -1):
            cordinate_one_day.append(getCordinate_Hotel(day['accommodation'], data_hotel))
        
        for attraction in day['morning_attractions']:
            if(attraction != -1):
                cordinate_one_day.append(getCordinate(attraction,data))
        for attraction in day['afternoon_attractions']:
            if(attraction != -1):
                cordinate_one_day.append(getCordinate(attraction,data))
        for attraction in day['night_attractions']:
            if(attraction != -1):
                cordinate_one_day.append(getCordinate(attraction,data))
                
        cordinates.append(cordinate_one_day)
    return cordinates

def getCordinate(id,data):
    for attraction in data:
        if attraction['business_id'] == id:
            return (attraction['latitude'], attraction['longitude'])

def getCordinate_Hotel(id,data_hotel):
    for hotel in data_hotel:
        if hotel['business_id'] == id:
            return (hotel['latitude'], hotel['longitude'])
        
def getDistanceMatrix(cordinates):
    #print(cordinates)
    n = len(cordinates)
    distance_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(i+1, n):
            distance_matrix[i][j] = distance_matrix[j][i] = ((cordinates[i][0]*1000 - cordinates[j][0]*1000)**2 + (cordinates[i][1]*1000 - cordinates[j][1]*1000)**2)**0.5
    return distance_matrix

def populateShortestDistanceOneDay(cordinates):
    shortest_distance_list = []
    shortest_distance_info_lists = []
    for oneday in cordinates:
        distance_matrix = getDistanceMatrix(oneday)
        n = len(distance_matrix)
        info_lists = []
        optimized_distance = totalCost(1, 0, n, distance_matrix,info_lists)
        shortest_distance_list.append(optimized_distance)
        shortest_distance_info_lists.append(info_lists)
    return shortest_distance_list, shortest_distance_info_lists

def totalCost(mask, pos, n, cost, info_lists):
    distance_list = []
    i_list = []
    # Base case: if all cities are visited, return the
    # cost to return to the starting city (0)
    if mask == (1 << n) - 1:
        return cost[pos][0]

    ans = sys.maxsize   

    # Try visiting every city that has not been visited yet
    for i in range(n):
        if (mask & (1 << i)) == 0: 
            i_list.append(i)
            # If city i is not visited, visit it and 
            #  update the mask
            distance_list.append(cost[pos][i] +
                      totalCost(mask | (1 << i), i, n, cost, info_lists))
        

    info_list = [pos,i_list, distance_list]
    info_lists.append(info_list)
    
    ans = min(distance_list)
    return ans

def populatePlannedDistanceOneDay(cordinates):
    planned_distance_list = []
    for oneday in cordinates:
        distance_matrix = getDistanceMatrix(oneday)
        #print(distance_matrix)
        distance = 0
        for i in range(len(distance_matrix)):
            if i == len(distance_matrix) - 1:
                j = 0
            else:
                j = i + 1
            distance += distance_matrix[i][j]
        planned_distance_list.append(distance)
    return planned_distance_list

def getDistanceGapRatio(shortest_distances_by_day, planned_distances_by_day):
    distance_gap = 0
    total_distance = 0
    for optimized_distance, planned_distance in zip(shortest_distances_by_day, planned_distances_by_day):
        gap = []
        gap = np.sum(np.array(planned_distance) - np.array(optimized_distance))
        distance_gap += gap
        
        total = np.sum(np.array(planned_distance))
        total_distance += total
        
    return distance_gap / total_distance

def getOptimizedOrder(shortest_distance_info_lists):


    order_list = []
    for day in shortest_distance_info_lists:

        if len(day) == 0:
            order_list.append([[0],[0]])
            continue

        pos = 0
        n = len(day[-1][1]) + 1
        #get a list of 1 to n
        candidates = list(range(n-1))
        #add 1 to the values
        candidates = [x+1 for x in candidates]

        moves = []

        while len(candidates) > 0:
            #find the last one in the lnfo_list
            for i in range(len(day)):
                if day[i][0] == pos and day[i][1] == candidates:
                    #print(day[i][0],day[i][1])
                    next_move = day[i][1][np.argmin(day[i][2])]
                    #print(next_move)
                    pos = next_move
                    moves.append(next_move)
                    #take next move out of candidates
                    candidates.remove(next_move)

        moves_reversed = moves[::-1]
        optimized_route = [[0] + moves, [0] + moves_reversed]
        order_list.append(optimized_route)

    return order_list
        
def getPositionDeviationRatio(shortest_order_by_day):
    total_places = 0
    total_deviation = 0
    for plan in shortest_order_by_day:
        for day in plan:
            n = len(day[0])
            total_places += n
            output_route = list(range(n))
            gap_1 = sum([1 if x != y else 0 for x,y in zip(output_route,day[0])])
            gap_2 = sum([1 if x != y else 0 for x,y in zip(output_route,day[1])])
            total_deviation += min(gap_1, gap_2)
    return total_deviation / total_places

def daywiseTSP(model,task, numPlan):
    shortest_distances_by_day = []
    planned_distances_by_day = []
    shortest_order_by_day = []

    with open(f'Output/{model}/evals/{task}.jsonl', 'r') as f:
        plans = [json.loads(line) for line in f]
    for i in range(numPlan):
        if (i%20 == 0):
            print("Mode: day wise. We are at plan ", i)
        plan = plans[i]['plan'] 
        
        # Failure rate related
        # prepare a result list to return
        # outofpool, missinginfo,
        # prepare the evaluation for each plan, search the business id
        plan_eval = prepareEval(plan)
        #print(plan_eval)

        #get the cordinates
        cordinates = populateCordinates(plan_eval, attractions, hotels)
        #print(cordinates)
        #one day shortest distance
        shortest_distance_list_each_day, shortest_distance_info_lists = populateShortestDistanceOneDay(cordinates)
        #print(shortest_distance_info_lists)
        shortest_distances_by_day.append(shortest_distance_list_each_day)
        
        shortest_order_list_each_day = getOptimizedOrder(shortest_distance_info_lists)
        
        shortest_order_by_day.append(shortest_order_list_each_day)
        #shortest_order_by_day(info_list)
        
        #one day planned distance
        planned_distance_list_each_day = populatePlannedDistanceOneDay(cordinates)
        planned_distances_by_day.append(planned_distance_list_each_day)

        #plan wise (multi day) optimization calculation

    #get distance gap ratio
    distance_gap_ratio = getDistanceGapRatio(shortest_distances_by_day, planned_distances_by_day)

    #position deviation ratio
    position_deviation_ratio = getPositionDeviationRatio(shortest_order_by_day)
    
    return distance_gap_ratio, position_deviation_ratio

def getHotelIndex(day,cordinates):
    hotel_index = 0
    if day > 0:
        for j in range(day):
            hotel_index += len(cordinates[j])
    return hotel_index

def totalCost_multiday(mask, pos, day, cordinates, n, visited, cost, info_lists, memo):
    visit_requirement = len(cordinates[day])
    distance_list = []
    i_list = []

    hotel_index = getHotelIndex(day,cordinates)
    # Base case: if all cities are visited, return the
    # cost to return to the starting city (0)

    if mask == (1 << n) - 1:
        return cost[pos][hotel_index]
    
    if memo[pos][mask] != -1:
        return memo[pos][mask]

    if visit_requirement == visited:
        for i in range(n):
            if (mask & (1 << i)) == 0: 
                i_list.append(i)
                distance_list.append(cost[hotel_index][i] + totalCost_multiday(mask | (1 << i), i, day + 1, cordinates, n, 2, cost, info_lists,memo))
        
        info_list = [pos,i_list, distance_list]
        info_lists.append(info_list)
        
        return min(distance_list) + cost[pos][hotel_index] # change this to the old hotel position
    
    # Try visiting every city that has not been visited yet
    for i in range(n):
        if (mask & (1 << i)) == 0: 

            i_list.append(i)
            # If city i is not visited, visit it and 
             #  update the mask
            distance_list.append(cost[pos][i] +
                      totalCost_multiday(mask | (1 << i), i, day, cordinates, n, visited + 1, cost, info_lists,memo))
        

    info_list = [pos,i_list, distance_list]
    info_lists.append(info_list)
    
    memo[pos][mask] = min(distance_list)

    return min(distance_list)

def getDistanceMatrix_by_plan(cordinates):
    #print(cordinates)
    n = 0
    for day in cordinates:
        for place in day:
            n+=1
    flattened = []
    for day in cordinates:
        for location in day:
            flattened.append(location)
    distance_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(i+1, n):
            distance_matrix[i][j] = distance_matrix[j][i] = ((flattened[i][0]*1000 - flattened[j][0]*1000)**2 + (flattened[i][1]*1000 - flattened[j][1]*1000)**2)**0.5
    return distance_matrix

def getOptimizedDistance_by_plan(cordinates,distance_matrix):
    n = len(distance_matrix)
    info_lists = []
    #newMask will have all the hotels as 1 before the function.
    newMask = 1
    index_list = list(range(len(cordinates) - 1))
    index_list = index_list[::-1]
    newMask = 1
    for i in index_list:
        newMask = (newMask << (len(cordinates[i]))) + 1
    memo = [[-1] * (1 << n) for _ in range(n)]
    optimized_distance = totalCost_multiday(newMask,0,0,cordinates,n,1,distance_matrix, info_lists,memo)
    return optimized_distance, info_lists

def getOptimizedOrder_by_plan(info_lists):
    pos = 0
    lookfor = info_lists[-1][1].copy()
    moves = []
    while len(lookfor) > 0:
        for record in info_lists:
            if(record[0] == pos and record[1] == lookfor):
                nextmove = record[1][np.argmin(record[2])]
                pos = nextmove
                moves.append(nextmove)
                lookfor.remove(pos)
    return moves

def getPlannedDistance_by_plan(cordinates, distance_matrix):
    distance = 0
    for i in range(len(distance_matrix)):
        if i == len(distance_matrix) - 1:
            j = len(distance_matrix) - len(cordinates[-1])
        else:
            j = i + 1
        distance += distance_matrix[i][j]
    return distance

def getDistanceGapRatio_by_plan(optimized_distances_by_plan, planned_distances_by_plan):
    gaps = np.array([])
    for optimized,planned in zip(optimized_distances_by_plan,planned_distances_by_plan):
        gap = planned - optimized
        gaps = np.append(gaps,gap)
    total_gap = np.sum(gaps)
    total_planned = np.sum(planned_distances_by_plan)
    ratio = total_gap / total_planned
    return ratio

def getDistanceGapRatio_by_plan(optimized_distances_by_plan, planned_distances_by_plan):
    gaps = np.array([])
    for optimized,planned in zip(optimized_distances_by_plan,planned_distances_by_plan):
        gap = planned - optimized
        gaps = np.append(gaps,gap)
    total_gap = np.sum(gaps)
    total_planned = np.sum(planned_distances_by_plan)
    ratio = total_gap / total_planned
    return ratio

def getClusterJumpRatio(optimized_orders_by_plan, cordinates_list):
    gaps = 0
    totals = 0
    for order, cordinates in zip(optimized_orders_by_plan,cordinates_list):
        days = []
        for _ in cordinates:
            days.append(len(_))
        days = np.array(days) - 1 #
        #print(days)
        optimzied_cluster_count = len(days) #
        #print(optimzied_cluster_count)

        totalplaces = optimzied_cluster_count + len(order)
        #print(totalplaces)
        candidates = list(range(totalplaces))
        #print(candidates)
        
        cluster_list = [] #
        for cand in candidates:
            if cand in order:
                index = order.index(cand)
                clusterNumber = getcluster(index,days)
                cluster_list.append(clusterNumber)
        #print(cluster_list)

        cluster_set =  makeClusterSet(cluster_list,days)
        #print(cluster_set)

        cluster_visited_count = 0
        for cluster in cluster_set:
            cluster_visited_count += len(cluster)
        #print(cluster_visited_count)

        gaps += cluster_visited_count - optimzied_cluster_count
        totals += optimzied_cluster_count
    #print(gaps,totals)
    return gaps/totals

def getcluster(index,days):
    clusterNumber = 0
    for day in days:
        index = index - day
        if index < 0:
            return clusterNumber
        else:
            clusterNumber += 1

def makeClusterSet(cluster_list,days):
    cluster_sets = []
    for day in days:
        cluster = []
        n = day
        while n > 0:
            cluster.append(cluster_list[0])
            cluster_list.pop(0)
            n -= 1
        cluster_sets.append(set(cluster))
    return cluster_sets

def planwiseTSP(model, task, planIndex):
    optimized_distances_by_plan = []
    optimized_orders_by_plan = []
    planned_distances_by_plan = []
    cordinates_list = []

    ave_att_per_day = 0
    total_days = 0
    total_att = 0

    with open(f'Output/{model}/evals/{task}.jsonl', 'r') as f:
        plans = [json.loads(line) for line in f]
    print(plans[planIndex]['index'])
    plan = plans[planIndex]['plan'] 

    for day in plan['itinerary']:
        print(f"day: {day['days']}, morning: {day['morning_attractions']}, afternoon: {day['afternoon_attractions']}, night: {day['night_attractions']}, hotel: {day['accommodation']}")
    # Failure rate related
    # prepare a result list to return
    # outofpool, missinginfo,
    # prepare the evaluation for each plan, search the business id
    plan_eval = prepareEval(plan)
    print(plan_eval)
    
    #get the cordinates
    cordinates = populateCordinates(plan_eval, attractions, hotels)
    cordinates_list.append(cordinates)
    total_days += len(cordinates)
    #print(len(cordinates))

    distance_matrix = getDistanceMatrix_by_plan(cordinates)
    
    optimized_distance, info_lists = getOptimizedDistance_by_plan(cordinates,distance_matrix)
    
    optimized_distances_by_plan.append(optimized_distance)
    
    optimized_order = getOptimizedOrder_by_plan(info_lists)
    print(optimized_order)
    total_att+=len(optimized_order)

    optimized_orders_by_plan.append(optimized_order)
    

    planned_distance = getPlannedDistance_by_plan(cordinates, distance_matrix)
    planned_distances_by_plan.append(planned_distance)

    distance_gap_ratio_by_plan = getDistanceGapRatio_by_plan(optimized_distances_by_plan, planned_distances_by_plan)
    cluster_jump_ratio_by_plan = getClusterJumpRatio(optimized_orders_by_plan, cordinates_list)
    ave_att_per_day = total_att / total_days
    return distance_gap_ratio_by_plan,cluster_jump_ratio_by_plan,ave_att_per_day

if __name__ == "__main__":
    #load datas
    with open ('Dataset/gpt4o/restaurants.jsonl', 'r') as file:
        restaurants = [json.loads(line.strip()) for line in file]

    with open ('Dataset/gpt4o/hotels.jsonl', 'r') as file:
        hotels = [json.loads(line.strip()) for line in file]

    with open ('Dataset/gpt4o/attractions.jsonl', 'r') as file:
        attractions = [json.loads(line.strip()) for line in file]

    modelList = ['gpt4o','mistral','llama318b']
    taskList = ['allDataNoRoute','allDataRouteOP','filteredDataRouteOP','toolUsePlans','baseData']

    #choose model and task
    model = modelList[0]
    task = taskList[3]
    print("==Model: ", model,"==")
    print("==Task: ", task,"==")

    #TSP
    #distance_gap_ratio, position_deviation_ratio = daywiseTSP(model,task,0)
    distance_ratio, cluster_ratio, att_per_day = planwiseTSP(model,task,0)
    print(distance_ratio,cluster_ratio,att_per_day)

==Model:  gpt4o ==
==Task:  toolUsePlans ==
1
day: 1, morning: [{'name': 'Please Touch Museum', 'address': '4231 Avenue of the Republic, Philadelphia, PA 19131'}], afternoon: [{'name': 'Shofuso Japanese House & Garden', 'address': 'Horticultural Dr and Lansdowne Dr, Philadelphia, PA 19131'}, {'name': 'Philadelphia Zoo', 'address': '3400 W Girard Ave, Philadelphia, PA 19104'}], night: [{'name': 'Spruce Street Harbor Park', 'address': '121 N Columbus Blvd, Philadelphia, PA 19106'}], hotel: {'name': 'The Rittenhouse Hotel', 'address': '210 W Rittenhouse Sq, Philadelphia, PA 19103'}
day: 2, morning: [{'name': 'Franklin Square', 'address': '200 N 6th St, Philadelphia, PA 19106'}], afternoon: [{'name': 'Museum of the American Revolution', 'address': '101 S 3rd St, Philadelphia, PA 19106'}, {'name': 'The Franklin Institute Science Museum', 'address': '222 N 20th St, Philadelphia, PA 19103'}], night: [{'name': 'Rittenhouse Square Park', 'address': '1800 Walnut St, Philadelphia, PA 19103'}], ho

In [3]:
import json

In [4]:
with open ('Output/gpt4o/plans/toolUseLogs.jsonl','r') as file:
    logs = [json.loads(line) for line in file]

In [8]:
logs[0]['log'][3]['observation']

'{\'Cluster_0\': [[\'Shane Confectionery\'], [\'The Constitutional Walking Tour\'], [\'Franklin Square\'], [\'Franklin Square Mini Golf\'], [\'Founding Footsteps\'], [\'The Big Bus Company\'], [\'The Betsy Ross House\'], [\'Museum of the American Revolution\'], [\'National Constitution Center\'], [\'National Liberty Museum\'], [\'National Museum of American Jewish History\'], [\'The African American Museum in Philadelphia\'], [\'Old City District\'], [\'Independence After Hours\'], [\'Independence Park Hotel, Bw Premier Collection\'], [\'Philadelphia Segway Tours By Wheel Fun Rentals\'], [\'Lokal Hotel\'], [\'Cherry Street Pier\'], [\'Philadelphia Sightseeing Tours & Transportation\'], [\'Bow Tie Tours\'], [\'Philadelphia Urban Adventures\'], ["Spirits of \'76 Ghost Tours"], [\'Benjamin Franklin Museum\'], [\'Philly Tour Hub\'], [\'Powel House\'], ["Penn\'s View Hotel"], [\'The Franklin Residences\'], [\'Tattooed Mom\'], [\'Historic Philadelphia, Inc\']], \'Cluster_1\': [[\'Morris Arbo